## RAG integration with Azure AI Search

### Installing ai search SDK

In [ ]:
%pip install azure-search-documents

### Setting in the variables with keys and credentials

In [ ]:
search_key="AZURE_AI_SEARCH_KEY"
search_endpoint="AZURE_AI_SEARCH_ENDPOINT"
index_name="INDEX_NAME"

### Creating AI Search Client

In [ ]:
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(search_key)

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient

search_client = SearchClient(endpoint=search_endpoint,
                      index_name=index_name,
                      credential=credential)

### Creating Dataframe and populating it with data from Azure AI Search

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Initialize Spark session (already initialized in Microsoft Fabric notebooks)
spark = SparkSession.builder.getOrCreate()

# Define schema for the DataFrame
schema = StructType([
    StructField("content", StringType(), True),
    StructField("metadata_storage_name", StringType(), True)
])

# Initialize an empty list to hold rows for the DataFrame
data = []

# Perform the search
results = search_client.search(
    query_type="simple",
    search_text="*",
    include_total_count=True
)

# Iterate through the results and extract fields
for result in results:
    data.append((
        result.get("content", ""),  # Extract content field
        result.get("metadata_storage_name", "")  # Extract metadata_storage_name field
    ))

# Create a Spark DataFrame
df = spark.createDataFrame(data, schema)

# Show the Spark DataFrame
df.show()